In [3]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold
from sklearn import metrics
import matplotlib.pyplot as plt
from functools import reduce
import math
from collections import Counter
import csv
from datetime import datetime
import re
import copy
import json
%matplotlib inline

In [4]:
def now():
    return str(datetime.now().time())[:8]
def pr(strToPrint):
    print(now() + ' '+ strToPrint)

from IPython.display import Audio
sound_file = 'beep.wav'
# pd.set_option('display.max_colwidth', -1)

## Importing data

Importing a sample of the dataset:

In [5]:
pickle_filename = os.path.join('data','head_100k_pickle.pkl')
tw = pd.read_pickle(pickle_filename)

Importing the whole dataset

In [6]:
columns_header = ['id', 'userId', 'createdAt', 'text', 'longitude', 'latitude', 'placeId',
                  'inReplyTo', 'source', 'truncated', 'placeLatitude', 'placeLongitude', 'sourceName', 'sourceUrl',
                 'userName', 'screenName', 'followersCount', 'friendsCount', 'statusesCount',
                 'userLocation']

filename = os.path.join('data','twex.tsv') # 'sample.tsv')
pr('Starting to read file... (3 min)')
# tw = pd.read_csv(filename, sep='\t', encoding='utf-8', escapechar='\\', names=columns_header,
#                       quoting=csv.QUOTE_NONE, na_values='N', header=None)

pr('File is loaded!')
# Audio(url=sound_file, autoplay=True)

17:11:13 Starting to read file...


C:\Users\Daniel\Anaconda3\envs\ADA-kernel\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0,1,2,3,5,6,8,9,10,11,12,13,14,15,16,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


17:13:48 File is loaded!


In [7]:
tw.head(2)

,id,userId,createdAt,text,longitude,latitude,placeId,inReplyTo,source,truncated,placeLatitude,placeLongitude,sourceName,sourceUrl,userName,screenName,followersCount,friendsCount,statusesCount,userLocation
0,9514097914,1.7341e+07,2010-02-23 05:55:51,Guuuuten Morgen! :-),7.43926,46.9489,NaN,NaN,197,NaN,NaN,NaN,TwitBird,http://www.nibirutech.com,Tilman Jentzsch,blickwechsel,586,508.0,9016.0,"Bern, Switzerland"
1,9514846412,7.19828e+06,2010-02-23 06:22:40,Still the best coffee in town — at La Stanza h...,8.53781,47.3678,NaN,NaN,550,NaN,NaN,NaN,Gowalla,http://gowalla.com/,Nico Luchsinger,halbluchs,1820,703.0,4687.0,"Zurich, Switzerland"


## Extrating hashtags

In [8]:
def extract_hashtags(text):
    ht_list = re.findall(r"#(\w+)", text)
    non_empty_hts = list(filter((lambda ht: ht != []), ht_list))
    lowerCharList = [ht.lower() for ht in non_empty_hts]
    return lowerCharList

In [9]:
pr('Making hashtags... (2 min)')
tw['hashtag'] = np.nan
tw.hashtag = tw.text.apply(lambda x: extract_hashtags(str(x)))
twh = tw.ix[tw.hashtag.apply(lambda x: len(x) != 0)]
pr('Done.')

17:13:48 Making hashtags...
17:15:40 Done.


In [10]:
twh.head(2)

,id,userId,createdAt,text,longitude,latitude,placeId,inReplyTo,source,truncated,...,placeLongitude,sourceName,sourceUrl,userName,screenName,followersCount,friendsCount,statusesCount,userLocation,hashtag
8,9519737890,1.46579e+07,2010-02-23 09:59:41,"Magic spells run off after midnight, I guess s...",6.13870,46.175,NaN,NaN,1,NaN,...,NaN,Twitter for iPhone,http://twitter.com/#!/download/iphone,Javier Belmonte,vichango,167,277.0,2885.0,"Geneva, Switzerland",[fb]
10,9521789689,9.96202e+06,2010-02-23 11:28:27,"Limitas of public transportation! No taxi, rai...",6.33641,46.4631,NaN,NaN,550,NaN,...,NaN,Gowalla,http://gowalla.com/,Thomas Winter,thwinter,1070,1359.0,3349.0,Hettlingen CH / SanJose Ca,[yam]


## Cleaning data and making date index

In [11]:
tw1 = twh.dropna(axis=0, how='any', subset=['text', 'createdAt'])
print('The data have been reduced from {} tweets to {} tweets.'.format(len(twh), len(tw1)))

The data have been reduced from 3875280 tweets to 3875280 tweets.


In [12]:
pr('Removing bad dates...')
twhCleanDate = tw1[tw1['createdAt'].str.len() == 19]
pr('Finished.')

17:15:43 Removing bad dates...
17:15:46 Finished.


In [13]:
pr('Starting to examine dates...')
import warnings
warnings.filterwarnings('ignore')
datetime_serie = twhCleanDate['createdAt'].convert_objects(convert_dates='coerce')
dateNotConvertible = datetime_serie[pd.isnull(datetime_serie)]
warnings.filterwarnings('default')
pr('There are {} dates that cannot be transformed.'.format(len(dateNotConvertible)))

17:15:47 Starting to examine dates...
17:15:49 There are 0 dates that cannot be transformed.


In [14]:
pr('Starting copy...') # (to avoid transformation problems)
tw5 = twhCleanDate.copy()
pr('Converting to datetime...')
tw5['createdAt'] = pd.to_datetime(twhCleanDate['createdAt'])
pr('Setting up new indices...')
tw5.index = tw5['createdAt']
pr('Deleting old "createdAt" column...')
del tw5['createdAt']
pr('Done!')
tw5.head(2)

17:15:49 Starting copy...
17:15:51 Converting to datetime...
17:15:55 Setting up new indices...
17:15:55 Deleting old "createdAt" column...
17:15:55 Done!


,id,userId,text,longitude,latitude,placeId,inReplyTo,source,truncated,placeLatitude,placeLongitude,sourceName,sourceUrl,userName,screenName,followersCount,friendsCount,statusesCount,userLocation,hashtag
createdAt,,,,,,,,,,,,,,,,,,,,
2010-02-23 09:59:41,9519737890,1.46579e+07,"Magic spells run off after midnight, I guess s...",6.13870,46.175,NaN,NaN,1,NaN,NaN,NaN,Twitter for iPhone,http://twitter.com/#!/download/iphone,Javier Belmonte,vichango,167,277.0,2885.0,"Geneva, Switzerland",[fb]
2010-02-23 11:28:27,9521789689,9.96202e+06,"Limitas of public transportation! No taxi, rai...",6.33641,46.4631,NaN,NaN,550,NaN,NaN,NaN,Gowalla,http://gowalla.com/,Thomas Winter,thwinter,1070,1359.0,3349.0,Hettlingen CH / SanJose Ca,[yam]


In [15]:
tw5['hashtag'][:6]

createdAt
2010-02-23 09:59:41               [fb]
2010-02-23 11:28:27              [yam]
2010-02-23 17:47:11          [24, vfb]
2010-02-23 18:19:03    [iphoneography]
2010-02-23 18:31:46     [partnermonth]
2010-02-24 06:09:23      [insider, fb]
Name: hashtag, dtype: object

## Let's put one hashtag per row

We will make a dataframe with one row = one hashtag. This will be done by going through the dataframe, and making in parallel a list of rows (with 1 hashtag per row) that needs to be added to the old dataframe.

In [16]:
addedHashtagsRowsList = []
def multiplyHashtagRows(row, columns):
    '''
    Examine each row. If there are multiple hashtags, it will return the first one.
    (so the first one will replace the list of hashtags in the df). Then for all the next ones,
    it will make a copy of the row in the addedHashtagsRowsList, (in a dictionary format).
    So this dictionary can in the end be transformed in a DF and added to the original DF.
    (The speed is increased a lot by doing it this way!)
    '''
    htList = row.hashtag
    if len(htList) > 1:
        ## Making the dictionary
        addedHashtag = {}
        addedHashtag['createdAt'] = row.name #the df index
        for col in columns:
            addedHashtag[col] = row[col]
        ## Copying the dict for each hashtag
        i = 1
        while i < len(htList) :
            deepCopy = copy.deepcopy(addedHashtag)
            deepCopy['hashtag'] = htList[i]
            addedHashtagsRowsList.append(deepCopy)
            i+=1
    return htList[0] # return the first hashtag

In [17]:
addedHashtagsRowsList = []
tw5_1 = tw5.copy()
pr('Multiplying the hashtag rows... (around 10 min)')
tw5_1['hashtag'] = tw5.apply(multiplyHashtagRows, args=[tw5.columns,], axis=1)
pr('Finished! {} rows will be added to the dataframe!'.format(len(addedHashtagsRowsList)))

17:15:56 Multiplying the hashtag rows... (around 30min)
17:25:15 Finished! 3350600 rows will be added to the dataframe!


In [18]:
pr('Starting to make the new dataframe with additionnal rows..')
addedHashtagsDf = pd.DataFrame(addedHashtagsRowsList)
addedHashtagsDf.set_index(['createdAt'], inplace=True)
pr('Starting to append the two df... Old df size = {}'.format(len(tw5_1)))
tw6 = tw5_1.append(addedHashtagsDf)
pr('Done! New df size = {}'.format(len(tw6)))

17:25:15 Starting to make the new dataframe with additionnal rows..
17:25:28 Starting to append the two df... Old df size = 3875280
17:25:34 Done! New df size = 7225880


In [19]:
print('Example hahshtag:')
tw6[tw6['hashtag'] == addedHashtagsRowsList[0]['hashtag']].head(3)

Example hahshtag:


,followersCount,friendsCount,hashtag,id,inReplyTo,latitude,longitude,placeId,placeLatitude,placeLongitude,screenName,source,sourceName,sourceUrl,statusesCount,text,truncated,userId,userLocation,userName
createdAt,,,,,,,,,,,,,,,,,,,,
2011-07-24 16:03:13,1121,763.0,vfb,95162125584039936,9.501397e+16,NaN,NaN,e401fb8eb4e7595a,47.5356,9.14004,eLd0raDo,14,Tweetbot for iOS,http://tapbots.com/tweetbot,6735.0,"@mikstweed Wie lange ist es her, dass der #vfb...",NaN,921241,"Kreuzlingen, TG, Switzerland",Markus Tressl
2012-07-29 16:15:54,1008,767.0,vfb,229611214802669570,NaN,47.6683,9.48077,7e00050f2f9230bc,47.6684,9.47092,jens_nagler,1,Twitter for iPhone,http://twitter.com/#!/download/iphone,1697.0,Man sollte ja nicht zu viel reininterpretieren...,NaN,5.71385e+07,Esslingen,Jens Nagler
2012-07-29 16:21:45,1008,767.0,vfb,229612686525231104,2.296124e+17,47.6682,9.48068,7e00050f2f9230bc,47.6684,9.47092,jens_nagler,1,Twitter for iPhone,http://twitter.com/#!/download/iphone,1697.0,"@LLcurly Das Besondere daran ist, dass nur zwe...",NaN,5.71385e+07,Esslingen,Jens Nagler


In [20]:
tw6.hashtag.head()

createdAt
2010-02-23 09:59:41               fb
2010-02-23 11:28:27              yam
2010-02-23 17:47:11               24
2010-02-23 18:19:03    iphoneography
2010-02-23 18:31:46     partnermonth
Name: hashtag, dtype: object

## Aggregation of rows by date

In [21]:
tw6.dropna(subset=['longitude'], inplace=True)
tw6.dropna(subset=['latitude'], inplace=True)

In [22]:
tw6.latitude = tw6.latitude.apply(float)
tw6.longitude = tw6.longitude.apply(float)

In [23]:
delimiter = '_$$$_'
str_join = lambda x: delimiter.join(x)

Function that applies to a dataframe will group each row by day and aggregate all its content:

In [24]:
def aggDate(df):
    groupedDf = df.groupby(df.index.map(lambda x: x.date)).agg({'text' : str_join,
                                                          'longitude' : np.median,
                                                          'latitude' : np.median,
                                                          'hashtag' : lambda x: x.iloc[0], ## the first occurence
                                                          'numberOfTweets' : 'count'})
    return groupedDf

In [47]:
pr('Making column number of tweets')
tw6['numberOfTweets'] = 1
pr('Starting group by hastag...')
gp = tw6.groupby('hashtag')
pr('Starting to put hashtag in dictionary... (around 50 min)')

count = 0
lengp = len(gp)
printingValue = int(lengp / 10)
dictionary = {}
for hashtag, df in gp:
    dictionary[hashtag] = aggDate(df)
    count += 1
    if count % printingValue == 0:
        pr("{:.0f}%".format(count/lengp*100))
pr('Finished operations! Dictionary with {} different hashtags.'.format(len(dictionary)))

20:18:47 Making column number of tweets
20:18:47 Starting group by hastag...
20:18:48 Starting to put hashtag in dictionary... (around 50 min)
20:19:46 10%
20:19:57 20%
20:20:08 30%
20:20:19 40%
20:20:31 50%
20:20:42 60%
20:20:53 70%
20:21:04 80%
20:21:15 90%
20:21:27 100%
20:21:27 Finished operations! Dictionary with 0 different hashtags.


In [26]:
# aggDate(gp.get_group('twibisg')).head(3)
# dictionary['twibisg'].head(3)
print('Dictionary with hashtags dataframes:')
dictionary[list(dictionary.keys())[5]].head(4)

Dictionary with hashtags dataframes:


,latitude,numberOfTweets,hashtag,longitude,text
2014-03-30,45.9060,1,oglio,10.2173,#sunday #morning #fiume #oglio #pistaciclabile...
2014-08-13,46.0730,1,oglio,10.3499,L'#oglio a #cedegolo #brescia questa sera non ...
2015-05-21,45.8786,1,oglio,10.1710,Sentiero fluviale dell'Oglio. #fiumeoglio #val...
2015-06-09,46.1185,1,oglio,10.3187,Futuri scenari camuni della pista ciclabile #c...


## Starting event detection

In [27]:
## Parameters of an event:
MIN_TOT_NB_TWEETS = 20 ## The hashtag must have happened at least this number of times to be considered.
MIN_NB_DAYS_WITH_HASHTAGS = 3 ## The hashtags must appear at least this number of different days to be considered.
MIN_NB_TWEETS_DURING_EVENT = 5 ## To be considered an event, the hashtag must happen at least this nb of times during the day.
THRESHOLD_ANOMALY_FACTOR = 2 ## The occurence of a hashtag during a single day must be above the mean by this FACTOR
                             ## multiplied by the std to be considered as an anomaly.

In [28]:
pr('Starting to compute {} dict items to detect event. (4 min)'.format(len(dictionary)))
nbOfEventDetected = 0
count = 0
printingValue = int(len(dictionary) / 10)
for [h,df] in dictionary.items():
    count += 1
    if count % printingValue == 0:
        pr("{:.0f}%".format(count/len(dictionary)*100))
    df['event'] = False
    if len(df) > MIN_NB_DAYS_WITH_HASHTAGS:
        if df['numberOfTweets'].sum() > MIN_TOT_NB_TWEETS:
            threshold = df['numberOfTweets'].mean() + THRESHOLD_ANOMALY_FACTOR * df['numberOfTweets'].std()
            df['event'] = df.numberOfTweets.apply(lambda x: x > threshold and x > MIN_NB_TWEETS_DURING_EVENT)
            nbOfEventDetected += len(df[df['event']])
pr('Finished! Number of events detected = {}'.format(nbOfEventDetected))

18:13:53 Starting to compute 607601 dict items to detect event.
18:14:16 10%
18:14:39 20%
18:15:02 30%
18:15:25 40%
18:15:48 50%
18:16:11 60%
18:16:34 70%
18:16:57 80%
18:17:37 90%
18:18:01 100%
18:18:01 Finished! Number of events detected = 22109


## Making single event dataframe

In [29]:
eventRowsList = []
def applyToMakeEventDf(row):
    if row.event:
        rowToAdd = {'date': row.name, 'hashtag': row.hashtag, 'text': row.text,
                    'longitude': row.longitude, 'latitude':row.latitude, 'numberOfTweets': row.numberOfTweets, }
        eventRowsList.append(rowToAdd)

In [30]:
eventRowsList = []
count = 0
printingValue = int(len(dictionary) / 10)

pr('Starting to make event df with {} dataframes. (around 6 min)'.format(len(dictionary)))
for h, df in dictionary.items():
    count += 1
    if count % printingValue == 0:
        pr("{:.0f}%".format(count/len(dictionary)*100))
    df.apply(applyToMakeEventDf, axis=1)

pr('Making new dataframe.')
new_events = pd.DataFrame(eventRowsList)
new_events.set_index(['date'], inplace=True)
pr('Finished! Dataframe with {} rows'.format(len(new_events)))

18:18:01 Starting to make event df with 607601 dataframes. (around 30 min)
18:18:35 10%
18:19:09 20%
18:19:44 30%
18:20:17 40%
18:20:51 50%
18:21:26 60%
18:22:00 70%
18:22:34 80%
18:23:09 90%
18:23:43 100%
18:23:43 Making new dataframe.
18:23:43 Finished! Dataframe with 22109 rows


In [31]:
print('Events dataframe:')
new_events.head(10)

Events dataframe:


,hashtag,latitude,longitude,numberOfTweets,text
date,,,,,
2014-06-01,wedding,47.19805,8.561180,6,A perfect day for a #wedding :) #rivadelgarda ...
2014-07-18,wedding,46.69090,7.116670,6,#wedding #friends #selfie #duckface @ Villa Cl...
2014-07-26,wedding,45.95815,8.339025,6,Mandatory #wedding #selfie _$$$_Wedding at a S...
2014-09-06,wedding,45.90510,8.531145,10,Невеста дирижирует:) #wedding #music #conducto...
2014-09-13,wedding,46.49730,9.840680,7,Such a funny wedding bus! #wedding #bus @ Lauf...
2014-12-21,wedding,46.51670,6.500000,9,#Wedding #Switzerland #Familly #Classy @ Morge...
2015-06-26,wedding,45.82080,8.806580,6,Matrimonio in corso #wedding #villaparadeisos ...
2015-07-25,wedding,45.81600,9.092790,12,• selfie con la sposa • \n#23 #luglio #2015 #s...
2015-08-29,wedding,46.20800,8.186690,6,Abflug zum #Wedding Weekend (@ EuroAirport Bas...


In [32]:
print('Linked dataframe of all days:')
dictionary[new_events.iloc[0].hashtag].head(10)

Linked dataframe of all days:


,latitude,numberOfTweets,hashtag,longitude,text,event
2010-10-12,47.4074,1,wedding,9.747370,@vincic we married 10-10 as well (1997) :) sti...,False
2011-07-30,47.2247,1,wedding,8.824080,Unfreiwillig eine Freinacht eingelegt. Das kan...,False
2011-08-20,47.2258,1,wedding,8.818800,Auf gehts an die Hochzeit. Bei schönsten Wette...,False
2011-08-31,45.8052,1,wedding,9.245610,@eroispaziali così a spanne azzarderei un -7 #...,False
2012-05-06,47.4628,1,wedding,8.957420,the wedding invitations r ready! #ilike #wedding,False
2012-05-10,47.5815,1,wedding,6.856990,"@BarackObama Congratulations from France, for ...",False
2012-05-19,45.8150,1,wedding,9.067470,chi ben beve.. #wedding @ Villa Saporiti http...,False
2012-05-25,47.4536,1,wedding,8.558090,Immer noch mit den Beatz von Jay-Z im Ohr und ...,False
2012-05-26,45.9843,2,wedding,7.738975,Je suis sansé me prepare pour me\nDiriger en d...,False
2012-06-05,46.1952,1,wedding,6.191870,After #wedding drinks ! (@ Les Secrets de Phil...,False


## Exporting data

In [33]:
e_df = new_events.copy()
e_df['date'] = e_df.index
e_df.index = [i for i in range (len(e_df))]
e_df.head(1)

,hashtag,latitude,longitude,numberOfTweets,text,date
0,wedding,47.19805,8.56118,6,A perfect day for a #wedding :) #rivadelgarda ...,2014-06-01


In [34]:
e_gb = e_df.groupby(e_df.date)

In [35]:
'latitude:{0}, longitude:{1}, tweets: [{2}], number_of_tweets : {3} '.format('a', 'b', 'c', 'd')

'latitude:a, longitude:b, tweets: [c], number_of_tweets : d '

In [36]:
tweet_unit_template = 'name:{0}, latitude : {1}, longitude : {2}, \
            tweets: [{3}], number_of_tweets : {4}'

data_template = 'data: [{0}]' # put 'tweet_unit, ...'
event_template = 'date:{0}, {1}' #put data to {1}

In [40]:
events = []
pr('Making event list...')
for date, df in e_gb:
        units = []
        for i in range (len(df)):
            ht = df.iloc[i]['hashtag']
            lat = df.iloc[i]['latitude']
            lon = df.iloc[i]['longitude']
            t_num = df.iloc[i]['numberOfTweets']
            tweets = df.iloc[i]['text'].split(delimiter)
            
            data_unit = { 'name': ht
                         , 'latitude' : lat
                         , 'longitude' : lon
                         , 'tweets' : tweets
                         , 'number_of_tweets' : str(t_num) }
            units.append(data_unit)
        
        date_dict = {}
        #date_json = 'date: {0}'.format(date)
        #data_json = data_template.format(','.join(units))
        curr_event = {'date': date.isoformat(), 'data': units}
        events.append(curr_event)

final_events = {'events' : events}
pr('Finished.')

19:21:47 Making event list...
19:22:05 Finished.


In [41]:
# final_events

In [42]:
pr('Exporting to json...')
with open('data/export.json', 'w') as f:
     json.dump(final_events, f)
pr('Export done.')

19:23:13 Exporting to json...
19:23:15 Export done.
